In [1]:
# import dependencies
from datetime import datetime
import backtrader as bt

In [23]:
class BollingerBandStrategy(bt.Strategy):
    
    params = (
            ('period', 20),
            ('std', 2),
            ('size', 25000)
    )
    
    def __init__(self):
        self.bollinger = bt.indicators.BollingerBands(period = self.p.period,
                                                     devfactor = self.p.std)
        
    def next(self):
        
        # if no open positions
        if not self.position:
            
            # open short position
            if self.data.close[0] > self.bollinger.lines.top:
                self.sell(size=self.p.size)
            
            # open long positions if needed
            if self.data.close[0] < self.bollinger.lines.bot:
                self.buy(size=self.p.size)
        
        else:
            
            # if we have opened a long position already...
            if self.position.size > 0:
                # close position when price crosses moving average
                self.sell(exectype=bt.Order.Limit,
                          price=self.bollinger.lines.mid[0],
                          size=self.p.size)
            else:
                self.buy(exectype=bt.Order.Limit,
                         price=self.bollinger.lines.mid[0],
                         size=self.p.size)
            

In [24]:
# main program
cerebro = bt.Cerebro()
cerebro.addstrategy(BollingerBandStrategy)

cerebro.adddata(bt.feeds.YahooFinanceCSVData(
        dataname='GBPUSD=X.csv',
        fromdate=datetime(2018, 11, 9),
        todate=datetime(2023, 11, 9)
        ))

cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)

results = cerebro.run()

# print metrics
print(f"Sharpe Ratio: {results[0].analyzers.sharperatio.get_analysis()['sharperatio']:.3f}")
print(f"Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%")
print(f"Max. Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")
print("Capital: $%.2f" % cerebro.broker.getvalue())

Sharpe Ratio: 1.173
Annual Return: 3.53%
Max. Drawdown: 13.47%
Capital: $11962.50
